### 1. General info of dataset GSE235923

This is the Jupyter Notebook for dataset GSE235923. Its dataset includes barcodes/features/matrix files for each sample.

Thus, we need to simply incorparate these barcodes/genes/matrix files and generate an AnnData object for each sample. 

In total, there are 31 acute myeloid leukemia (AML) samples.

<span style="color:green">**[Dx]**</span> samples from diagnosis

<span style="color:green">**[EOI]**</span> samples from end of induction

<span style="color:green">**[R]**</span> samples from relapse 

In [1]:
# Environment setup
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as anndata
import scipy

### 2. AnnData object of each sample

<span style="color:red">**IMPORTANT:**</span> rename files to get rid of prefixes 

1. `barcodes.tsv.gz`: cell barcodes, which go into `.obs`
2. `features.tsv.gz`: gene names, `.var`
3. `matrix.mtx.gz`: the expression matrix, `.X`

In [2]:
from pathlib import Path

# Specify directory paths
data_directory = Path('/scratch/user/s4543064/xiaohan-john-project/data/GSE235923')
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE235923')

# Loop through all files in the directory
for sample_directory in data_directory.iterdir():
    sample_name = sample_directory.stem
    sample_h5ad = sample_name + '_uni.h5ad'

    sample = sc.read_10x_mtx(
    sample_directory,
    var_names='gene_symbols',  
    cache=False
    )

    # Create an observation metric info to store related features
    obs_metrics = pd.DataFrame(index=sample.obs_names) ## Get the identifiers
    obs_metrics['cancer_type'] = 'acute_myeloid_leukemia'
    obs_metrics['dataset'] = 'GSE235923'
    obs_metrics['tissue'] = 'bone_marrow'
    obs_metrics['uni_barcode'] = obs_metrics['dataset'] + '_' + obs_metrics.index.astype(str)
    
    sample.obs = obs_metrics
    sample.obs.set_index("uni_barcode", drop=False, inplace=True)
    print(sample)

    # save the anndata object
    output_path = write_directory / sample_h5ad
    sample.write_h5ad(output_path, compression="gzip")

AnnData object with n_obs × n_vars = 3495 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 3758 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 2214 × 36601
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 5041 × 36601
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 2413 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 9588 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 3057 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_

### 3. Confirmation of created AnnData objects

In [3]:
from pathlib import Path

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE235923')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample = anndata.read_h5ad(file)
    print(sample)

AnnData object with n_obs × n_vars = 4268 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 5440 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 1699 × 36601
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 2679 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 3758 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 3442 × 36601
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 4430 × 36601
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_

In [5]:
sample.obs

,cancer_type,dataset,tissue,uni_barcode
uni_barcode,,,,
GSE235923_AAACCTGAGAGCTATA-1,acute_myeloid_leukemia,GSE235923,bone_marrow,GSE235923_AAACCTGAGAGCTATA-1
GSE235923_AAACCTGAGGGTTCCC-1,acute_myeloid_leukemia,GSE235923,bone_marrow,GSE235923_AAACCTGAGGGTTCCC-1
GSE235923_AAACCTGGTTCTGTTT-1,acute_myeloid_leukemia,GSE235923,bone_marrow,GSE235923_AAACCTGGTTCTGTTT-1
GSE235923_AAACGGGCACCCTATC-1,acute_myeloid_leukemia,GSE235923,bone_marrow,GSE235923_AAACGGGCACCCTATC-1
GSE235923_AAACGGGCAGGACCCT-1,acute_myeloid_leukemia,GSE235923,bone_marrow,GSE235923_AAACGGGCAGGACCCT-1
...,...,...,...,...
GSE235923_TTTGTCAAGAAGGTGA-1,acute_myeloid_leukemia,GSE235923,bone_marrow,GSE235923_TTTGTCAAGAAGGTGA-1
GSE235923_TTTGTCAAGTGAACAT-1,acute_myeloid_leukemia,GSE235923,bone_marrow,GSE235923_TTTGTCAAGTGAACAT-1
GSE235923_TTTGTCACAAGTTAAG-1,acute_myeloid_leukemia,GSE235923,bone_marrow,GSE235923_TTTGTCACAAGTTAAG-1
